### Read train set and decode data

In [1]:
import base64
import codecs
import nltk

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\egalkin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
text_corpus = []
Y = []

with codecs.open('kaggle_train_data_tab.csv', 'r', 'utf8') as file:
    file.readline()
    i = 1
    for line in file:
        processed_line = line.split('\t')
        decoded_html = base64.b64decode(processed_line[3]).decode('utf8', errors='replace')
        Y.append(int(processed_line[1]))
        text_corpus.append(decoded_html)
        
test_text_corpus = []
Y_Test = []
Ids = []

with codecs.open('kaggle_test_data_tab.csv', 'r') as file:
    file.readline()
    for line in file:
        processed_line = line.split('\t')
        Ids.append(int(processed_line[0]))
        Y_Test.append(int(processed_line[1]))
        test_text_corpus.append(base64.b64decode(processed_line[3]).decode('utf8', errors='replace'))        
    

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian') + stopwords.words('english')

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
vectorizer = TfidfVectorizer(stop_words = stop_words, ngram_range=(1,3), max_features = 53125)
vectorizer.fit(text_corpus + test_text_corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=53125,
                min_df=1, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                            'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                            'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                            'по', 'только', 'ее', 'мне', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [39]:
X = vectorizer.transform(text_corpus)

In [40]:
from sklearn import svm

In [41]:
clf = svm.SVC()
clf.fit(X, Y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [42]:
X_Test = vectorizer.transform(test_text_corpus)

In [43]:
prediction = clf.predict(X_Test)

In [44]:
with codecs.open('submission.csv', 'w', 'utf8') as submission:
    submission.write('Id,Prediction\n')
    for i in range(len(prediction)):
        submission.write(f'{Ids[i]},{prediction[i]}\n')

In [45]:
print(X.shape)
print(X_Test.shape)

(7044, 53125)
(16039, 53125)
